In [1]:
# Arxiv Research
# Tools Creation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


```markdown
1. Wikipedia ve Arxiv Araçlarının Oluşturulması
Amaç: Wikipedia ve Arxiv sorguları için gerekli araç ve yardımcı sınıfları projeye dahil eder.
WikipediaQueryRun & ArxivQueryRun: Wikipedia ve Arxiv üzerinde arama yapabilen araçlar.
WikipediaAPIWrapper & ArxivAPIWrapper: Bu araçların arka planda kullandığı API sarmalayıcıları.
```

In [ ]:
# Used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


'wikipedia'

```markdown
2. Wikipedia Aracının Oluşturulması
Amaç: Wikipedia'da arama yapacak bir araç oluşturmak.
WikipediaAPIWrapper: Wikipedia'dan en fazla 1 sonuç ve 200 karakterlik içerik çeker.
WikipediaQueryRun: Bu sarmalayıcıyı kullanarak Wikipedia sorguları yapar.
wiki.name: Aracın adını gösterir.

```

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


```markdown
3. Arxiv Aracının Oluşturulması
Amaç: Arxiv'de arama yapacak bir araç oluşturmak.
ArxivAPIWrapper: Arxiv'den en fazla 1 sonuç ve 250 karakterlik içerik çeker.
ArxivQueryRun: Bu sarmalayıcıyı kullanarak Arxiv sorguları yapar.
print(arxiv.name): Aracın adını ekrana yazdırır.

```

In [5]:
tools = [wiki,arxiv]


```markdown
4. Araçların Listeye Eklenmesi
Amaç: Oluşturulan araçları bir listeye ekleyerek daha sonra topluca kullanmak.

```

In [6]:
# Custom tools (Rag Tool)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


```markdown
5. Kendi Dokümanınızdan Arama Yapacak Aracın Oluşturulması (RAG Tool)
Amaç: Belirli bir web sayfasından (ör: LangSmith dokümantasyonu) veri çekip, bu veriyi vektör veritabanına kaydetmek ve arama yapılabilir hale getirmek.
WebBaseLoader: Web sayfası içeriğini çeker.
RecursiveCharacterTextSplitter: İçeriği küçük parçalara böler.
OpenAIEmbeddings: Parçaları vektörlere dönüştürür.
FAISS: Vektörleri saklayan ve arama yapabilen veritabanı.
retriever: Bu veritabanında arama yapmayı sağlayan nesne.
```

In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000288F5DD13D0>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

```markdown
6. Retriever Tool'un Oluşturulması
Amaç: Yukarıda oluşturulan retriever'ı bir araca dönüştürmek.
create_retriever_tool: Retriever'ı, agent'ın kullanabileceği bir araca çevirir.
retriever_tool.name: Aracın adını gösterir
```

In [9]:
tools = [wiki,arxiv,retriever_tool]

```markdown
7. Tüm Araçların Listeye Eklenmesi
Amaç: Artık üç farklı kaynaktan (Wikipedia, Arxiv, LangSmith dokümanı) arama yapabilen araçlar tek bir listede.

```

In [10]:
# Run all this tools with Agents and LLM Models
# Tools, LLM -> AgentExecutor

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os 

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key = groq_api_key,model_name ="Llama3-8b-8192")

```markdown
8. LLM ve Ortam Değişkenlerinin Ayarlanması
Amaç: Büyük dil modelini (LLM) ve gerekli API anahtarlarını ayarlamak.
load_dotenv(): .env dosyasından anahtarları yükler.
ChatGroq: Groq firmasının Llama3 modelini kullanır.
```

In [11]:
# Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

```markdown
9. Prompt Template'in Hazırlanması
Amaç: Agent'ın kullanacağı hazır prompt şablonunu almak.
hub.pull: LangChain Hub'dan hazır prompt çeker.


hub Nedir?
hub, LangChain ekosisteminde, topluluk tarafından hazırlanmış ve paylaşılan hazır prompt şablonlarını, zincirleri (chains), agent yapılarını ve diğer bileşenleri merkezi bir depodan kolayca çekmenizi sağlayan bir modüldür.
Yani, kendi promptunuzu sıfırdan yazmak yerine, başkalarının hazırladığı ve test ettiği şablonları hızlıca kullanabilirsiniz.
Kodun Açıklaması
hub.pull("hwchase17/openai-functions-agent") fonksiyonu, LangChain Hub'dan "hwchase17/openai-functions-agent" isimli hazır bir agent prompt şablonunu indirir.
Bu şablon, OpenAI fonksiyonlarıyla çalışan bir agent için optimize edilmiştir ve genellikle araç kullanan (tool-using) agent'lar için önerilir.
Sonuç olarak, prompt değişkeni artık bu hazır şablonun içeriğini tutar ve agent oluştururken doğrudan kullanılabilir.
Neden Kullanılır?
Zaman kazandırır: Sıfırdan prompt yazmak yerine, topluluk tarafından denenmiş bir şablon kullanırsınız.
Standartlaştırır: Farklı projelerde aynı şablonu tekrar tekrar kullanabilirsiniz.
Güncel kalır: Hub’daki şablonlar topluluk tarafından güncellenebilir.
Özet
hub ile, LangChain Hub’daki hazır bileşenleri kolayca projenize dahil edersiniz.
Bu satır, bir agent’ın nasıl davranacağını ve araçları nasıl kullanacağını belirleyen, test edilmiş bir prompt şablonunu indirir ve kullanıma hazır hale getirir.
Kısacası:
hub = LangChain’in hazır şablon ve zincir deposu,
pull = oradan istediğin bileşeni çekmek için kullanılır.
```

In [12]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

```markdown

burada openai agent promptunu kullanıyoruz ama llm llama , bu hazır şablonu kullandığımız için create_openai_tools_agent fonksiyonunu kullanmak zorundayız değil mi?



Evet, doğru anladınız.
prompt = hub.pull("hwchase17/openai-functions-agent") satırı ile OpenAI fonksiyon tabanlı bir agent için hazırlanmış bir prompt şablonu kullanıyorsunuz. Bu şablon, araç kullanan (tool-using) agent yapısı için optimize edilmiştir ve LangChain'de OpenAI uyumlu agent fonksiyonlarıyla çalışacak şekilde tasarlanmıştır.

Bu nedenle, bu prompt ile birlikte agent oluşturmak için:

fonksiyonunu kullanmak gerekir.

Burada LLM olarak Llama (ör. Groq üzerinden) kullanmanızda teknik olarak bir engel yoktur; çünkü LangChain, OpenAI API uyumlu başka modelleri de bu fonksiyonla çalıştırabiliyor. Ancak, prompt ve agent yapısı OpenAI fonksiyon formatına göre hazırlandığı için, bu fonksiyonu kullanmak zorundasınız.

Özet:
Eğer bu hazır OpenAI fonksiyon agent promptunu kullanıyorsanız, create_openai_tools_agent fonksiyonunu kullanmak zorundasınız. LLM'iniz OpenAI API uyumlu olduğu sürece (Groq Llama gibi) bu yapı çalışır.


```

```markdown
10. Agent'ın Oluşturulması
Amaç: LLM, araçlar ve prompt ile bir agent (yapay zeka asistanı) oluşturmak.
Agent, hangi soruda hangi aracı kullanacağını otomatik belirler.
```

In [13]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executer


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

```markdown
11. AgentExecutor ile Agent'ın Çalıştırılması
Amaç: Agent'ı ve araçları bir araya getirip, dışarıdan gelen soruları işleyip cevap dönen bir yapı oluşturmak.
verbose=True: Adım adım işlemleri gösterir.
```

In [14]:
agent_executer.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': "The Langsmith platform provides a robust set of tools for building, monitoring, and evaluating Large Language Models (LLMs) applications. It offers observability features, prompt engineering, and evaluation capabilities to help developers create high-quality and production-grade LLM applications.\n\nSome key features of Langsmith include:\n\n1. Observability: Langsmith provides LLM-native observability, allowing developers to gain meaningful insights into their application's performance and debug any issues.\n2. Evaluation: Langsmith offers a range of evaluation tools and metrics to help developers test and optimize their LLM applications.\n3. Prompt Engineering: Langsmith provides a set of tools and features to help developers find the perfect prompts for their LLM applications, making it easier to iterate on prompts and collaborate with others.\n4. Deployment: Langsmith allows developers to deploy their LLM applications on the LangGrap

```markdown
Agent’lar, LangChain ve benzeri framework’lerde, birden fazla aracı (tool) ve büyük dil modelini (LLM) bir araya getirip, karmaşık görevleri otomatik olarak çözebilen akıllı yapay zeka “yürütücüleridir”.

Kısaca Agent nedir?
Agent, kendisine verilen doğal dildeki bir soruyu/parçayı analiz eder.
Hangi aracı (ör. Wikipedia, Arxiv, özel doküman arama) kullanması gerektiğine kendi karar verir.
Gerekirse birden fazla aracı art arda veya birlikte kullanabilir.
Sonuçları birleştirip, kullanıcıya anlamlı bir cevap döndürür.
Ne işe yarar?
Çoklu bilgi kaynağı: Farklı kaynaklardan (web, vektör veritabanı, API, Wikipedia, Arxiv vs.) otomatik bilgi toplar.
Otomasyon: Hangi adımda hangi aracı kullanacağını, hangi sırayla çalışacağını kendi belirler.
Akıllı yönlendirme: Sadece bir LLM’den cevap almak yerine, gerektiğinde dış veri kaynaklarına başvurur.
Karmaşık görevler: Sadece soru-cevap değil, zincirleme işlemler, veri toplama, özetleme gibi görevleri de üstlenebilir.
Örnek
Kullanıcı “Atatürk kimdir?” diye sorduğunda:

Agent önce Wikipedia aracını kullanmaya karar verir, oradan bilgi çeker.
“LangSmith nedir?” sorusunda ise kendi özel doküman arama aracını kullanır.
Tüm bu süreci, sizin kod yazmanıza gerek kalmadan, kendi başına yönetir.
Özet
Agent = Akıllı, çok kaynaklı, otomatik karar veren yapay zeka yürütücüsü
Kullanıcıdan gelen soruları analiz eder, uygun aracı seçer, çalıştırır ve cevabı üretir.
LangChain’de agent’lar, LLM ve araçları birleştirerek çok daha esnek ve güçlü uygulamalar geliştirmenizi sağlar.


```

```markdown
12. Sorguların Agent'a Gönderilmesi
Amaç: Kullanıcıdan gelen doğal dildeki soruları agent'a iletmek ve cevap almak.
Agent, sorunun içeriğine göre uygun aracı seçip, cevabı üretir.
```

In [15]:
agent_executer.invoke({"input":"What is machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genIt seems like we've already covered the basics of machine learning. Based on the information provided, it appears that machine learning is a subfield of artificial intelligence that involves developing and studying statistical algorithms that can learn from data and make predictions or decisions.

Machine learning is a rapidly growing field with many applications in various industries, including but not limited to natural language p

{'input': 'What is machine learning?',
 'output': "It seems like we've already covered the basics of machine learning. Based on the information provided, it appears that machine learning is a subfield of artificial intelligence that involves developing and studying statistical algorithms that can learn from data and make predictions or decisions.\n\nMachine learning is a rapidly growing field with many applications in various industries, including but not limited to natural language processing, computer vision, speech recognition, and predictive analytics."}

In [16]:
agent_executer.invoke({"input":"Who is Atatürk?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Mustafa Kemal Atatürk'}`


Page: Mustafa Kemal Atatürk
Summary: Mustafa Kemal Atatürk (c. 1881 – 10 November 1938) was a Turkish field marshal and revolutionary statesman who was the founding father of the Republic of Turkey, sMustafa Kemal Atatürk was a Turkish field marshal and revolutionary statesman who was the founding father of the Republic of Turkey. He played a key role in the Turkish War of Independence and served as the first President of Turkey from 1923 until his death in 1938.

> Finished chain.


{'input': 'Who is Atatürk?',
 'output': 'Mustafa Kemal Atatürk was a Turkish field marshal and revolutionary statesman who was the founding father of the Republic of Turkey. He played a key role in the Turkish War of Independence and served as the first President of Turkey from 1923 until his death in 1938.'}

```markdown
Özet:
Bu kodlar, Wikipedia, Arxiv ve kendi dokümanınızdan arama yapabilen, LLM destekli bir yapay zeka asistanı oluşturur. Her kod parçası, bu asistanın bir parçasını inşa eder ve sonunda doğal dilde sorulara otomatik olarak doğru kaynaktan cevap verebilen bir sistem elde edilir.
```

```markdown
Eğer Wikipedia veya Arxiv dışındaki istediğiniz bir web sitesini doküman haline getirip arama yapılabilir bir araca dönüştürmek isterseniz, aşağıdaki adımları izlemelisiniz:

1. Web Sitesinin İçeriğini Yükleyin
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://istediğiniz-site.com")
docs = loader.load()

Burada "https://istediğiniz-site.com" kısmını kendi istediğiniz web sitesiyle değiştirin.
docs değişkeni, sitenin metin içeriğini tutar.

2. Dokümanları Parçalara Bölün

from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

Büyük metinleri küçük parçalara ayırır, böylece daha verimli arama yapılabilir.

3. Embedding ve Vektör Veritabanı Oluşturun
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()

Her bir doküman parçası vektörlere dönüştürülür ve FAISS veritabanında saklanır.
retriever ile bu veritabanında arama yapılabilir.

4. Retriever'ı Bir Araca Dönüştürün

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "my-custom-site-search",
    "Search any information from my custom site"
)

Artık bu araç, agent tarafından kullanılabilir.


5. Aracı tools listesine ekleyin

tools = [wiki, arxiv, retriever_tool]

Böylece agent, yeni eklediğiniz web sitesinde de arama yapabilir.

Özet:
Wikipedia ve Arxiv dışındaki herhangi bir web sitesini de aynı yöntemle doküman haline getirip, arama yapılabilir bir araca dönüştürebilirsiniz. Sadece WebBaseLoader'a istediğiniz sitenin adresini verin ve yukarıdaki adımları uygulayın.

```